In [3]:
from tqdm import tqdm
from itertools import combinations
from pathlib import Path
import sys
import warnings
import logging
from pprint import pformat

import numpy as np
import pandas as pd
from pmdarima.arima import ARIMA, auto_arima
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.regularizers import l1_l2
import dynamic_yaml
import yaml
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

sys.path.append("/tf/correlation-coef-predict/ywt_library")
import data_generation
from data_generation import data_gen_cfg
from ywt_arima import arima_model, arima_err_logger_init
from stl_decompn import stl_decompn
from corr_property import calc_corr_ser_property

with open('../config/data_config.yaml') as f:
    data = dynamic_yaml.load(f)
    data_cfg = yaml.full_load(dynamic_yaml.dump(data))

warnings.simplefilter("ignore")
logging.basicConfig(level=logging.INFO)
matplotlib_logger = logging.getLogger("matplotlib")
matplotlib_logger.setLevel(logging.ERROR)
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False
# logger_list = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
# print(logger_list)


# %load_ext pycodestyle_magic
# %pycodestyle_on --ignore E501
logging.info(pformat(data_cfg, indent=1, width=100, compact=True))

[<Logger concurrent.futures (INFO)>, <Logger concurrent (INFO)>, <Logger asyncio (INFO)>, <Logger tornado.access (INFO)>, <Logger tornado (INFO)>, <Logger tornado.application (INFO)>, <Logger tornado.general (INFO)>, <Logger stack_data.serializing (INFO)>, <Logger stack_data (INFO)>, <Logger prompt_toolkit.buffer (INFO)>, <Logger prompt_toolkit (INFO)>, <Logger parso.python.diff (INFO)>, <Logger parso.python (INFO)>, <Logger parso (INFO)>, <Logger parso.cache (INFO)>, <Logger pkg_resources.extern.packaging.tags (INFO)>, <Logger pkg_resources.extern.packaging (INFO)>, <Logger pkg_resources.extern (INFO)>, <Logger pkg_resources (INFO)>, <Logger IPKernelApp (DEBUG)>, <Logger tqdm.cli (INFO)>, <Logger tqdm (INFO)>, <Logger sklearn (INFO)>, <Logger setuptools.extern.packaging.tags (INFO)>, <Logger setuptools.extern.packaging (INFO)>, <Logger setuptools.extern (INFO)>, <Logger setuptools (INFO)>, <Logger setuptools.config._apply_pyprojecttoml (INFO)>, <Logger setuptools.config (INFO)>, <Logg

# Prepare data

## Data implement & output setting & testset setting

In [ ]:
res_dir = Path('./results/')
res_dir.mkdir(parents=True, exist_ok=True)

# setting of output files
save_corr_data = True
save_arima_resid_data = True
# data implement setting
data_implement = "SP500_20082017_FREQ_CLUSTER_LABEL_1"  # watch options by operate: print(data_cfg["DATASETS"].keys())
# test set setting
test_items_setting = "-test_test"  # -test_test|-test_all
# data split period setting, only suit for only settings of Korean paper
data_split_setting = "-data_sp_test2"
# lstm weight setting
lstm_weight_setting = "SP500_20082017_KS_HYPER_LSTM"  # watch options by operate: print(data_cfg["LSTM_WEIGHT"].keys())

In [ ]:
# data loading & implement setting
dataset_df = pd.read_csv(data_cfg["DATASETS"][data_implement]['FILE_PATH'])
dataset_df = dataset_df.set_index('Date')
all_set = list(dataset_df.columns)  # all data
train_set = data_cfg["DATASETS"][data_implement]['TRAIN_SET']
test_set = data_cfg['DATASETS'][data_implement]['TEST_SET'] if data_cfg['DATASETS'][data_implement].get('TEST_SET') else [p for p in all_set if p not in train_set]  # all data - train data
logging.info(f"===== len(train_set): {len(train_set)}, len(all_set): {len(all_set)}, len(test_set): {len(test_set)} =====")

# test items implement settings
items_implement = test_set if test_items_setting == "-test_test" else all_set
logging.info(f"===== len(test set): {len(items_implement)} =====")

lstm_weight_filepath = data_cfg["LSTM_WEIGHT"][lstm_weight_setting]["FILE_PATH"]
lstm_weight_name = data_cfg["LSTM_WEIGHT"][lstm_weight_setting]["LSTM_WEIGHT_NAME"]
logging.info(f"===== LSTM weight:{lstm_weight_name} =====")

# setting of name of output files and pictures title
output_file_name = data_cfg["DATASETS"][data_implement]['OUTPUT_FILE_NAME_BASIS'] + test_items_setting + lstm_weight_name
fig_title = data_implement + test_items_setting + lstm_weight_name + data_split_setting
logging.info(f"===== file_name basis:{output_file_name}, fig_title basis:{fig_title} =====")
# display(dataset_df)
# display(all_set)

In [ ]:
# corr_window = 100
# corr_stride = 100
# data_length = int(len(dataset_df)/corr_window)*corr_window
# corr_ind = list(range(99, 2400, corr_stride))  + list(range(99+20, 2500, corr_stride)) + \
#            list(range(99+40, 2500, corr_stride)) + list(range(99+60, 2500, corr_stride)) + \
#            list(range(99+80, 2500, corr_stride))  # only suit for settings of paper

# corr_series_length = int((data_length-corr_window)/corr_stride)
# corr_series_length_paper = 21  # only suit for settings of paper
# data_diverse_stride = 20  # only suit for settings of paper
# data_diverse_start_dates = [99 + d for d in range(0,100,data_diverse_stride)]  # only suit for settings of paper
# data_diverse_start_dates

In [ ]:
# def gen_unseen_data_corr(item_pair: list, data_split_setting:str = "_test2") -> "pd.DataFrame, pd.Series | pd.DataFrame":
#     tmp_corr = dataset_df[item_pair[0]].rolling(window=corr_window).corr(dataset_df[item_pair[1]])
#     tmp_corr = tmp_corr.iloc[data_diverse_start_dates[0]::corr_stride]
#     if data_split_setting == "_test2":
#         corr_series = tmp_corr[3:24] # correspond to test2_dataset of original paper
#     elif data_split_setting == "_test1" :
#         corr_series = tmp_corr[2:23] # correspond to test1_dataset of original paper
#     elif data_split_setting == "_dev":
#         corr_series = tmp_corr[1:22] # correspond to dev_dataset of original paper
#     elif data_split_setting == "_train":
#         corr_series = tmp_corr[:21] # correspond to train_dataset of original papaer 
#     unseen_data_df = pd.DataFrame(corr_series).T
#     unseen_data_df.index = [f"{item_pair[0]} & {item_pair[1]}"]
#     return unseen_data_df

In [ ]:
# display(gen_unseen_data_corr([items_implement[0], items_implement[1]], data_split_setting="_test2"))
# display(gen_unseen_data_corr([items_implement[0], items_implement[1]], data_split_setting="_test1"))
# display(gen_unseen_data_corr([items_implement[0], items_implement[1]], data_split_setting="_dev"))
# display(gen_unseen_data_corr([items_implement[0], items_implement[1]], data_split_setting="_train"))

## Load or Create Correlation Data

In [ ]:
corr_data_dir = Path(data_cfg["DIRS"]["PIPELINE_DATA_DIR"])/f"{output_file_name}-corr_data"
corr_data_dir.mkdir(parents=True, exist_ok=True)
data_length = int(len(dataset_df)/data_gen_cfg["CORR_WINDOW"])*data_gen_cfg["CORR_WINDOW"]
corr_ser_len_max = int((data_length-data_gen_cfg["CORR_WINDOW"])/data_gen_cfg["CORR_STRIDE"])
max_data_div_start_add = 0  # In the Korea paper, each pair has 5 corr_series(due to diversifing train data).
                            # BUT we only need to take one, so take 0 as arg.
corr_ind = []

# DEFAULT SETTING: data_gen_cfg["DATA_DIV_STRIDE"] == 20, data_gen_cfg["CORR_WINDOW"]==100, data_gen_cfg["CORR_STRIDE"]==100
data_end_init = corr_ser_len_max * data_gen_cfg["CORR_STRIDE"]
for i in range(0, max_data_div_start_add+1, data_gen_cfg["DATA_DIV_STRIDE"]):
    corr_ind.extend(list(range(data_gen_cfg["CORR_WINDOW"]-1+i, data_end_init+bool(i)*data_gen_cfg["CORR_STRIDE"], data_gen_cfg["CORR_STRIDE"])))  # only suit for settings of paper

train_df_path = corr_data_dir/f"{output_file_name}-corr_train.csv"
dev_df_path = corr_data_dir/f"{output_file_name}-corr_dev.csv"
test1_df_path = corr_data_dir/f"{output_file_name}-corr_test1.csv"
test2_df_path = corr_data_dir/f"{output_file_name}-corr_test2.csv"
all_corr_df_paths = dict(zip(["train_df", "dev_df", "test1_df", "test2_df"],
                             [train_df_path, dev_df_path, test1_df_path, test2_df_path]))
if all([df_path.exists() for df_path in all_corr_df_paths.values()]):
    corr_datasets = [pd.read_csv(df_path).set_index("items") for df_path in all_corr_df_paths.values()]
else:
    corr_datasets = data_generation.gen_train_data(items_implement, raw_data_df=dataset_df, corr_ser_len_max=corr_ser_len_max, corr_df_paths=all_corr_df_paths, corr_ind=corr_ind, max_data_div_start_add=max_data_div_start_add, save_file=save_corr_data)

In [ ]:
if data_split_setting == "-data_sp_test2":
    corr_dataset = corr_datasets[3]
print(corr_datasets[0].shape, corr_datasets[1].shape, corr_datasets[2].shape, corr_datasets[3].shape)
display(corr_dataset.iloc[0,::])
display(corr_dataset.head())

# ARIMA model

In [ ]:
arima_result_dir = Path(data_cfg["DIRS"]["PIPELINE_DATA_DIR"])/f"{output_file_name}-arima_res"
arima_result_dir.mkdir(parents=True, exist_ok=True)
arima_result_path_basis = arima_result_dir/f'{output_file_name}.csv'
arima_result_types = ["-arima_output", "-arima_resid", "-arima_model_info"]
arima_result_paths = []
arima_err_logger_init(Path(os.path.abspath(''))/f"results")

for arima_result_type in arima_result_types:
    arima_result_paths.append(arima_result_dir/f'{output_file_name}{arima_result_type}{data_split_setting}.csv')

if all([df_path.exists() for df_path in arima_result_paths]):
    arima_output_df, arima_resid_df, arima_model_info_df = [pd.read_csv(arima_result_path, index_col="items") for arima_result_path in arima_result_paths]
else:
    arima_output_df, arima_resid_df, arima_model_info_df = arima_model(corr_dataset, arima_result_path_basis=arima_result_path_basis, data_split_setting=data_split_setting, save_file=save_arima_resid_data)

In [ ]:
# def arima_model(dataset: "pd.DataFrame", portfolio: list, overview: bool = False) -> ("np.array", "pd.DataFrame", str):
#     model_110 = ARIMA(order=(1, 1, 0), out_of_sample_size=10, mle_regression=True, suppress_warnings=True)
#     model_011 = ARIMA(order=(0, 1, 1), out_of_sample_size=10, mle_regression=True, suppress_warnings=True)
#     model_111 = ARIMA(order=(1, 1, 1), out_of_sample_size=10, mle_regression=True, suppress_warnings=True)
#     model_211 = ARIMA(order=(2, 1, 1), out_of_sample_size=10, mle_regression=True, suppress_warnings=True)
#     model_210 = ARIMA(order=(2, 1, 0), out_of_sample_size=10, mle_regression=True, suppress_warnings=True)
#     # model_330 = ARIMA(order=(3, 3, 0), out_of_sample_size=0, mle_regression=True, suppress_warnings=True)

#     # model_dict = {"model_110": model_110, "model_011": model_011, "model_111": model_111}
#     model_dict = {"model_110": model_110, "model_011": model_011, "model_111": model_111, "model_211": model_211, "model_210": model_210}
#     tested_models = []
#     arima_model = None
#     find_arima_model = False
#     arima_attr_list = ["aic", "arparams", "aroots", "maparams", "maroots", "params", "pvalues"]
#     arima_infos = dict(zip(arima_attr_list, [None]*len(arima_attr_list)))
#     for _, corr_series in dataset.iterrows():
#         while not find_arima_model:
#             try:
#                 for model_key in model_dict:
#                     if model_key not in tested_models:
#                         test_model = model_dict[model_key].fit(corr_series[:-1])  # only use first 20 corrletaion coefficient to fit ARIMA model
#                         if arima_model is None:
#                             arima_model = test_model
#                             arima_model_name = model_key
#                         elif arima_model.aic() <= test_model.aic():
#                             pass
#                         else:
#                             arima_model = test_model
#                             arima_model_name = model_key
#                     tested_models.append(model_key)
#             except Exception:
#                 if len(model_dict)-1 != 0:
#                     del model_dict[model_key]
#                 else:
#                     err_logger.error(f"fatal error, {portfolio} doesn't have appropriate arima model\n", exc_info=True)
#                     raise NotImplementedError(f"fatal error, {portfolio} doesn't have appropriate arima model\n")
#             else:
#                 # model_dict = {"model_110": model_110, "model_011": model_011, "model_111": model_111, "model_211": model_211, "model_210": model_210, "model_330": model_330}
#                 model_dict = {"model_110": model_110, "model_011": model_011, "model_111": model_111, "model_211": model_211, "model_210": model_210}
#                 tested_models.clear()
#                 find_arima_model = True
#         try:
#             arima_pred = list(arima_model.predict(n_periods=1))
#         except Exception:
#             err_logger.error(f"{portfolio} in {data_split_setting} be predicted by {arima_model_name}(its aic:{arima_model.aic()}) getting error:\n", exc_info=True)
#             raise NotImplementedError(f"{portfolio} in {data_split_setting} be predicted by {arima_model_name}(its aic:{arima_model.aic()}) getting error\n")
#         else:
#             arima_pred_in_sample = list(arima_model.predict_in_sample())
#             arima_pred_in_sample = [np.mean(arima_pred_in_sample[1:])] + arima_pred_in_sample[1:]
#             arima_output = arima_pred_in_sample + arima_pred
#             arima_output = np.clip(np.array(arima_output), -1, 1)

#             arima_resid = pd.Series(np.array(corr_series) - arima_output).iloc[:-1]

#             for attr in arima_infos.keys():
#                 try:
#                     arima_infos[attr] = getattr(arima_model, attr)()
#                 except AttributeError:
#                     pass
#         finally:
#             find_arima_model = False
#     if overview:
#         plt.plot(arima_output, label="arima_pred")
#         plt.plot(dataset.T, label="data")
#         plt.plot(arima_resid, label="res")
#         plt.legend()
#         plt.show()
#         plt.close()

#     return arima_output, arima_resid, arima_model_name, *[v for k, v in sorted(arima_infos.items(), key=lambda x:x[0])]

In [ ]:
# unseen_data_corr_df = gen_unseen_data_corr([items_implement[0], items_implement[1]], data_split_setting="_test2")
# arima_pred, residual, arima_model_name, arima_aic, arima_arparams, arima_aroots, arima_maparams, arima_maroots, arima_params, arima_pvalues = arima_model(unseen_data_corr_df, [items_implement[0], items_implement[1]], overview=True)
# display(len(arima_pred), len(residual))
# display(arima_pred, residual)

# LSTM

In [ ]:
def double_tanh(x):
    return (tf.math.tanh(x) * 2)


lstm_model = load_model(lstm_weight_filepath, custom_objects={'double_tanh':double_tanh})
lstm_model.summary()

# Hybrid model

In [ ]:
lstm_input = arima_resid_df.iloc[::,:-1].values.reshape(-1, 20, 1)
lstm_pred = lstm_model.predict(lstm_input)
lstm_pred = pd.DataFrame(lstm_pred, index=arima_resid_df.index, columns=["lstm_pred"])

In [ ]:
# res_list = []
# unseen_data_corr_df_concat = pd.DataFrame()
# unseen_data_arima_resid_concat = pd.DataFrame()
# count = 0
# for items in tqdm(combinations(items_implement, 2)):
#     unseen_data_corr_df = gen_unseen_data_corr(items, data_split_setting=data_split_setting)
#     try:
#         # arima_pred, residual, arima_model_name, arima_aic, arima_arparams, arima_aroots, arima_maparams, arima_maroots, arima_params, arima_pvalues = arima_model(unseen_data_corr_df, items)
#         pass
#     except NotImplementedError:
#         continue
#     else:
#         unseen_res = residual.values.reshape((-1, 20, 1))
#         lstm_pred = lstm_model.predict(unseen_res)
#         # season_period, stl_resid, stl_trend_std, coef_reg_trend = stl_decompn(unseen_data_corr_df.iloc[0, :])
#         items_res_dic = {"items": f"{items[0]} & {items[1]}",
#                          "corr_ser_mean": unseen_data_corr_df.mean(axis=1)[0],
#                          "corr_ser_std": unseen_data_corr_df.std(axis=1)[0],
#                          "corr_season_period": season_period,
#                          "corr_stl_resid": stl_resid,
#                          "corr_stl_trend_std": stl_trend_std,
#                          "corr_trend_coef": coef_reg_trend,
#                          "arima_model": arima_model_name,
#                          "arima_aic": arima_aic,
#                          "arima_arparams": arima_arparams,
#                          "arima_aroots": arima_aroots,
#                          "arima_maparams": arima_maparams,
#                          "arima_maroots": arima_maroots,
#                          "arima_params": arima_params,
#                          "arima_pvalues": arima_pvalues,
#                          "lstm_pred": lstm_pred[0][0],
#                          "arima_pred": arima_pred[-1],
#                          "ground_truth": unseen_data_corr_df.iloc[0, -1]}

#         res_list.append(items_res_dic)
#         unseen_data_corr_df['items'] = f"{items[0]} & {items[1]}"
#         unseen_data_corr_df_concat = pd.concat([unseen_data_corr_df_concat, unseen_data_corr_df])
#         residual['items'] = f"{items[0]} & {items[1]}"
#         unseen_data_arima_resid_concat = pd.concat([unseen_data_arima_resid_concat, residual], axis=1)

# if save_raw_corr_data:
#     unseen_data_corr_df_concat = unseen_data_corr_df_concat.set_index('items')
#     unseen_data_corr_df_concat.to_csv(f"./results/{output_file_name}-raw_corr.csv", index=True)

# if save_arima_resid_data:
#     unseen_data_arima_resid_concat = unseen_data_arima_resid_concat.T.set_index('items')
#     unseen_data_arima_resid_concat.to_csv(f"./results/{output_file_name}-arima_resid.csv", index=True)

## Results post-processing

In [ ]:
def res_df_postprocess(merge_dfs: list) -> "pd.DataFrame":

    tmp_df = pd.DataFrame(columns=["items"])
    for merge_df in merge_dfs:
        tmp_df = pd.merge(tmp_df, merge_df, on="items", how='outer')
    else:
        results_df = tmp_df.reset_index(drop=True)

    results_df["hybrid_model_pred"] =  results_df["arima_pred"] + results_df["lstm_pred"]
    results_df["error"] = results_df["ground_truth"] - results_df["hybrid_model_pred"]
    results_df["absolute_err"] = results_df["error"].abs()
    results_df['arima_pred_dir'] = np.sign(results_df['ground_truth'] * results_df['arima_pred'])
    results_df['arima_err'] = results_df['ground_truth'] - results_df['arima_pred']
    results_df["lstm_compensation_dir"] = np.sign(results_df['arima_err']) * np.sign(results_df['lstm_pred'])
    quantile_mask = np.logical_and(results_df['error'] < np.quantile(results_df['error'], 0.75), results_df['error'] > np.quantile(results_df['error'], 0.25)).tolist()
    # display(np.quantile(res_df['error'], 0.75), np.quantile(res_df['error'], 0.25))
    results_df['high_pred_performance'] = quantile_mask
    results_df['items[0]'] = results_df.apply(lambda row:row['items'].split(" & ")[0], axis=1)
    results_df['items[1]'] = results_df.apply(lambda row:row['items'].split(" & ")[1][:-2], axis=1)
    results_df = results_df.set_index("items")

    return results_df


# stl_decompn(corr_datasets[0].iloc[0,::], overview=True)
corr_property_dir = Path(data_cfg["DIRS"]["PIPELINE_DATA_DIR"])/f"{output_file_name}-corr_property"
corr_property_dir.mkdir(parents=True, exist_ok=True)
corr_property_df_path = corr_property_dir/f"{output_file_name}{data_split_setting}-corr_series_property.csv"

if corr_property_df_path.exists():
    corr_property_df = pd.read_csv(corr_property_df_path)
else:
    corr_property_df = calc_corr_ser_property(corr_dataset=corr_dataset, corr_property_df_path=corr_property_df_path)

ground_truth = corr_dataset.iloc[::, -1]
ground_truth.name = "ground_truth"
arima_pred = arima_output_df.iloc[::, -1]
arima_pred.name = "arima_pred"
merge_dfs = [arima_model_info_df, corr_property_df, arima_pred, lstm_pred, ground_truth]

res_df = res_df_postprocess(merge_dfs)
res_df.to_csv(res_dir/f"{output_file_name}{data_split_setting}-res.csv")

# def res_df_postprocess(target_df: pd.core.frame.DataFrame) -> None:
#     target_df["hybrid_model_pred"] =  target_df["arima_pred"] + target_df["lstm_pred"]
#     target_df["error"] = target_df["ground_truth"] - target_df["hybrid_model_pred"]
#     target_df["absolute_err"] = target_df["error"].abs()
#     target_df['arima_pred_dir'] = np.sign(target_df['ground_truth'] * target_df['arima_pred'])
#     target_df['arima_err'] = target_df['ground_truth'] - target_df['arima_pred']
#     target_df["lstm_compensation_dir"] = np.sign(target_df['arima_err']) * np.sign(target_df['lstm_pred'])
#     quantile_mask = np.logical_and(res_df['error'] < np.quantile(res_df['error'], 0.75), res_df['error'] > np.quantile(res_df['error'], 0.25)).tolist()
#     display(np.quantile(res_df['error'], 0.75), np.quantile(res_df['error'], 0.25))
#     target_df['high_pred_performance'] = quantile_mask
#     target_df['items[0]'] = target_df.apply(lambda row:row['items'].split(" & ")[0], axis=1)
#     target_df['items[1]'] = target_df.apply(lambda row:row['items'].split(" & ")[1], axis=1)


# res_df_postprocess(res_df)
# display(res_df)

# Display results

In [ ]:
res_df = pd.read_csv(res_dir/f"{output_file_name}{data_split_setting}-res.csv", index_col=["items"])
display(res_df.columns)
display(res_df.head())

In [ ]:
def plot_exploration(target_df: pd.core.frame.DataFrame, title: str) -> None:
    fig, axes = plt.subplots(figsize=(20, 20), nrows=7, ncols=2, sharex=False, sharey=False, dpi=100)
    s0 = axes[0, 0]
    s0.set_title("ABS_err violin")
    sns.violinplot(y=target_df["absolute_err"], ax=s0)
    s1 = axes[0, 1]
    s1.set_title("Err violin")
    sns.violinplot(y=target_df["error"], ax=s1)
    s2 = axes[1, 0]
    s2.set_title("ABS_err hist")
    target_df['absolute_err'].hist(bins=[b/10 for b in range(11)], ax=s2)
    s3 = axes[1, 1]
    s3.set_title("Err hist")
    target_df['error'].hist(bins=[b/10 for b in range(-10, 11)], ax=s3)
    s4 = axes[2, 0]
    s4.set_title("LSTM_compensation_dir count")
    sns.countplot(x="lstm_compensation_dir", data=target_df, ax=s4)
    s5 = axes[2, 1]
    s5.set_title("LSTM_compensation_dir count groupby ARIMA_pred_dir")
    df_gb = target_df.groupby(['arima_pred_dir', 'lstm_compensation_dir']).size().unstack(level=1)
    df_gb.plot(kind='bar', ax=s5)
    s6 = axes[3, 0]
    s6.set_title("ARIMA_model prediction Err violin group by LSTM_compensation_dir")
    sns.violinplot(x=target_df["lstm_compensation_dir"], y=target_df["arima_err"], ax=s6)
    s8 = axes[4, 0]
    s8.set_title("ARIMA_model prediction magnitude group by LSTM_compensation_dir")
    sns.violinplot(x=target_df["lstm_compensation_dir"], y=target_df["arima_pred"], ax=s8)
    s9 = axes[4, 1]
    s9.set_title("LSTM compensation magnitude group by LSTM_compensation_dir")
    sns.violinplot(x=target_df["lstm_compensation_dir"], y=target_df["lstm_pred"], ax=s9)
    s10 = axes[5, 0]
    s10.set_title("Correlation magnitude in last period group by LSTM_compensation_dir")
    sns.violinplot(x=target_df["lstm_compensation_dir"], y=target_df["ground_truth"], ax=s10)
    s11 = axes[5, 1]
    s11.set_title("Hybrid Err violin group by LSTM_compensation_dir")
    sns.violinplot(x=target_df["lstm_compensation_dir"], y=target_df["error"], ax=s11)
    s12 = axes[6,0]
    s12.set_title("LSTM_compensation_dir pie with wrong ARIMA_pred_dir")
    df_gb.loc[df_gb.index==-1, :].squeeze().plot(kind="pie", autopct='%1.1f%%', ax=s12)
    s13 = axes[6,1]
    s13.set_title("LSTM_compensation_dir pie with correct ARIMA_pred_dir")
    df_gb.loc[df_gb.index==1, :].squeeze().plot(kind="pie", autopct='%1.1f%%', ax=s13)

    fig.suptitle(f"{title}_basic_exploration")
    plt.tight_layout()
    plt.savefig(f"./results/hybrid_prediction_analysis_{title}.png")
    plt.show()
    plt.close()

In [ ]:
def plot_exploration_pred_perform(target_df: pd.core.frame.DataFrame, title: str) -> None:
    fig, axes = plt.subplots(figsize=(20, 20), nrows=6, ncols=2, sharex=False, sharey=False, dpi=100)
    s1 = axes[0, 0]
    s1.set_title("LSTM_compensation_dir count groupby prediction performance")
    df_gb = target_df.groupby(['high_pred_performance', 'lstm_compensation_dir']).size().unstack(level=1)
    df_gb.plot(kind='bar', ax=s1)
    s2 = axes[0, 1]
    s2.set_title("ARIMA_model prediction magnitude group by prediction performance")
    sns.violinplot(x=target_df["high_pred_performance"], y=target_df["arima_pred"], ax=s2)
    s3 = axes[1, 0]
    s3.set_title("LSTM compensation magnitude group by prediction performance")
    sns.violinplot(x=target_df["high_pred_performance"], y=target_df["lstm_pred"], ax=s3)
    s4 = axes[1, 1]
    s4.set_title("Correlation magnitude in last period group by prediction performance")
    sns.violinplot(x=target_df["high_pred_performance"], y=target_df["ground_truth"], ax=s4)
    s5 = axes[2, 0]
    s5.set_title("Correlation series mean groupby prediction performance")
    sns.violinplot(x=target_df['high_pred_performance'], y=target_df["corr_ser_mean"], ax=s5)
    s6 = axes[2, 1]
    s6.set_title("Correlation series std groupby prediction performance")
    sns.violinplot(x=target_df['high_pred_performance'], y=target_df["corr_ser_std"], ax=s6)
    s7 = axes[3, 0]
    s7.set_title("Correlation series stl_period group by prediction performance")
    sns.violinplot(x=target_df["high_pred_performance"], y=target_df["corr_stl_period"], ax=s7)
    s8 = axes[3, 1]
    s8.set_title("Correlation series stl_residual group by prediction performance")
    sns.violinplot(x=target_df["high_pred_performance"], y=target_df["corr_stl_resid"], ax=s8)
    s9 = axes[4, 0]
    s9.set_title("Correlation series stl_trend_std group by prediction performance")
    sns.violinplot(x=target_df["high_pred_performance"], y=target_df["corr_stl_trend_std"], ax=s9)
    s10 = axes[4, 1]
    s10.set_title("Correlation series stl_trend_coef group by prediction performance")
    sns.violinplot(x=target_df["high_pred_performance"], y=target_df["corr_stl_trend_coef"], ax=s10)
    s11 = axes[5, 0]
    s11.set_title("ARIMA_pred_dir count groupby prediction performance")
    df_gb = target_df.groupby(['high_pred_performance', 'arima_pred_dir']).size().unstack(level=1)
    df_gb.plot(kind='bar', ax=s11)

    fig.suptitle(F"{title}_groupby prediction")
    plt.tight_layout()
    plt.savefig(f"./results/hybrid_prediction_analysis_groupby_pred_perform_{title}.png")
    plt.show()
    plt.close()

In [ ]:
def plot_stock_freq(target_df: pd.core.frame.DataFrame, title: str) -> None:
    stocks_show_freq = target_df.loc[target_df['high_pred_performance'] == True, ['items[0]','items[1]']].stack().value_counts().to_dict()
    plt.figure(figsize=(80, 10), dpi=100)
    plt.bar(range(len(stocks_show_freq)), list(stocks_show_freq.values()))
    plt.xticks(range(len(stocks_show_freq)), list(stocks_show_freq.keys()), rotation=60)
    plt.title(F"{title}_items appearence frequence")
    plt.savefig(f"./results/items_appearence_frequence_{title}.png")
    plt.show()
    plt.close()

In [ ]:
display(f"mse :{(res_df['error']**2).mean()}",
        f"std of square_err :{(res_df['error']**2).std()}",
        f"rmse :{np.sqrt((res_df['error']**2).mean())}",
        f"mae : {res_df['absolute_err'].mean()}",
        f"std of abs_err: {res_df['absolute_err'].std()}",
        f"sklearn mse: {mean_squared_error(res_df['ground_truth'], res_df['hybrid_model_pred'])}")
display("-"*50)
display(f"mse of ARIMA :{(res_df['arima_err']**2).mean()}",
        f"std of square_err ARIMA :{(res_df['arima_err']**2).std()}",
        f"rmse of ARIMA :{np.sqrt((res_df['arima_err']**2).mean())}",
        f"sklearn mse of ARIMA: {mean_squared_error(res_df['ground_truth'], res_df['arima_pred'])}")

In [ ]:
plot_exploration(res_df, fig_title)
plot_exploration_pred_perform(res_df, fig_title)
plot_stock_freq(res_df, fig_title)